# blocs

In [1]:
import pandas as pd
import re
import requests

In [2]:
BLOCS = [
    ['vote_type', 'mail'],
    ['vote_type', 'eday'],
    ['pgh_region', 'nside'],
    ['pgh_region', 'somon'],
    ['pgh_region', 'dt_oak_haz'],
    ['pgh_region', 'w14'],
    ['pgh_region', 'rem_ee'],
]

In [3]:
def get_vote_type_pct(vote_type, candidates):
    r = requests.get(
        'https://results.enr.clarityelections.com/PA/Allegheny/109361/277443/json/%s.json' % vote_type,
        headers={
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
      })
    r = r.json()

    vote_type_sum = [p for p in r['Contests'] if p['A'] == '-1'][0]
    election_vote_type_sum = vote_type_sum['V'][vote_type_sum['C'].index(election_id)]
    return [
        election_vote_type_sum[0]/sum(election_vote_type_sum),
        election_vote_type_sum[1]/sum(election_vote_type_sum),
        sum(election_vote_type_sum[(len(candidates) - 1):])/sum(election_vote_type_sum)
    ]

### 2021 Pittsburgh mayoral primary

In [4]:
### Set candidate names, election ID
candidates = ['Gainey', 'Peduto', 'Other']
election_id = '0032'

pghmayoral21 = pd.DataFrame(columns=['class', 'type'])
pghmayoral21['pct_gain21'] = ''
pghmayoral21['pct_pedu21'] = ''
pghmayoral21['pct_othe21'] = ''

In [5]:
### Get mail, eday results
pghmayoral21.loc[len(pghmayoral21.index)] = ['vote_type', 'mail'] + get_vote_type_pct('Absentee', candidates)

pghmayoral21.loc[len(pghmayoral21.index)] = ['vote_type', 'eday'] + get_vote_type_pct('Election_Day', candidates)

In [6]:
### Get precinct results
r = requests.get(
    'https://results.enr.clarityelections.com/PA/Allegheny/109361/277443/json/ALL.json',
    headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
  })
r = r.json()

tempdf = pd.DataFrame(columns=(['precinct'] + candidates))

for precinct in r['Contests']:
    if election_id in precinct['C']:
        precinct_results = precinct['V'][precinct['C'].index(election_id)]
        precinct_results = precinct_results[0:(len(candidates) - 1)] + [sum(precinct_results[(len(candidates) - 1):])]
        tempdf.loc[len(tempdf.index)] = [precinct['A']] + precinct_results

tempdf = tempdf.head(-1)

In [7]:
PRECINCT_REGEX_PGH = 'PITTSBURGH WARD (\d+).*'
def group_pgh_precinct(precinct):
    ward = int(re.match(PRECINCT_REGEX_PGH, precinct).group(1))

    if ward == 14:
        return 'w14'
    elif ward in range(1, 7) or ward == 15:
        return 'dt_oak_haz'
    elif ward in range(6, 15):
        return 'rem_ee'
    elif ward in range(16, 21) or ward in range(27, 33):
        return 'somon'
    elif ward in range(21, 27):
        return 'nside'

tempdf['type'] = tempdf.precinct.apply(group_pgh_precinct)

In [8]:
tdf = pd.pivot_table(tempdf, index='type', values=candidates, aggfunc='sum').reset_index()
tdf['class'] = 'pgh_region'

tdf['total'] = tdf.sum(axis=1)
for candidate in candidates:
    tdf['pct_%s21' % candidate[0:4].lower()] = tdf[candidate] / tdf.total

/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_63385/375207897.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tdf['total'] = tdf.sum(axis=1)


In [9]:
pghmayoral21 = pd.concat([pghmayoral21, tdf[pghmayoral21.columns]])

### Results

In [10]:
dfblocs = pd.DataFrame(BLOCS, columns=['class', 'type'])
pd.merge(dfblocs, pghmayoral21, on=['class', 'type'])

,class,type,pct_gain21,pct_pedu21,pct_othe21
0,vote_type,mail,0.357575,0.526308,0.116116
1,vote_type,eday,0.525235,0.312933,0.161832
2,pgh_region,nside,0.483487,0.368371,0.148141
3,pgh_region,somon,0.305441,0.407443,0.287116
4,pgh_region,dt_oak_haz,0.509474,0.385675,0.104851
5,pgh_region,w14,0.441891,0.518149,0.039959
6,pgh_region,rem_ee,0.617377,0.311485,0.071138
